In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
%load_ext nb_black
%load_ext nb_mypy

In [ ]:
import nbimporter
from AIBaseClass import ChessAI
from Exercise04AI import Exercise04AI
from Exercise06AI import Exercise06AI
from Exercise07AI import Exercise07AI

# Aufgabe 08: Minimax mit Alpha-Beta-Pruning, Memoisierung, Progressive Deepening und Singular Value Extension

Dieses Notebook erweitert den Minimax-Algorithmus um die Singular Value Extension.

In [ ]:
import chess
from typing import Any


class Exercise08AI(Exercise07AI):
    """Chooses middle game moves using minimax algorithm, alpha-beta-pruning,
    memoization and progressive deepening and the quiescence search"""

    def __init__(self, **kwargs) -> None:
        super().__init__(**kwargs)
        self.real_depth: int = 0

    def reset(self) -> None:
        """Resets all internal variables"""
        super().reset()
        self.real_depth = 0

In [ ]:
from typing import Any, Callable


class Exercise08AI(Exercise08AI):  # type: ignore
    @staticmethod
    def memoize_minimax(minimax: Callable):
        def minimax_memoized(
            self,
            board: chess.Board,
            depth: int,
            current_evaluation: int,
            alpha: int = -Exercise08AI.LIMIT,
            beta: int = Exercise08AI.LIMIT,
            real_depth: int = -1,
        ):
            key = Exercise04AI.get_key(board, depth)
            self.stats[-1]["cache_tries"] += 1

            if key in self.cache:
                self.stats[-1]["cache_hits"] += 1
                return self.get_from_cache(
                    minimax,
                    key,
                    board,
                    depth,
                    current_evaluation,
                    alpha,
                    beta,
                    real_depth,
                )
            result = minimax(
                self, board, depth, current_evaluation, alpha, beta, real_depth
            )
            self.stats[-1]["max_depth"] = max(result[2], self.stats[-1]["max_depth"])
            self.store_in_cache(key, result, alpha, beta)
            return result

        return minimax_memoized

    def store_in_cache(self, key: tuple, result: tuple, alpha: int, beta: int) -> None:
        """Stores the result of a minimax computation in the cache."""
        evaluation, move, eval_depth = result
        if evaluation < alpha:
            self.cache[key] = ("≤", evaluation, move, eval_depth)
        elif evaluation <= beta:
            self.cache[key] = ("=", evaluation, move, eval_depth)
        else:
            self.cache[key] = ("≥", evaluation, move, eval_depth)

    def get_from_cache(
        self,
        minimax: Callable,
        key: tuple,
        board: chess.Board,
        depth: int,
        current_eval: int,
        alpha: int,
        beta: int,
        real_depth: int,
    ) -> tuple:
        """Gets a result from the cache if possible."""
        flag, evaluation, move, eval_depth = self.cache[key]
        if flag == "=":
            return evaluation, move, eval_depth
        elif flag == "≤":
            if evaluation <= alpha:
                return evaluation, move, eval_depth
            elif evaluation < beta:
                result = minimax(
                    self, board, depth, current_eval, alpha, evaluation, real_depth
                )
                self.store_in_cache(key, result, alpha, evaluation)
                return result
            else:
                result = minimax(
                    self, board, depth, current_eval, alpha, beta, real_depth
                )
                self.store_in_cache(key, result, alpha, beta)
                return result
        else:
            if evaluation <= alpha:
                result = minimax(
                    self, board, depth, current_eval, alpha, beta, real_depth
                )
                self.store_in_cache(key, result, alpha, beta)
                return result
            elif evaluation < beta:
                result = minimax(
                    self, board, depth, current_eval, evaluation, beta, real_depth
                )
                self.store_in_cache(key, result, evaluation, beta)
                return result
            else:
                return evaluation, move, eval_depth

In [ ]:
def debug_minimax(minimax: Callable):
    """Prints the Minimax Game Tree."""

    def minimax_debug(
        self,
        board: chess.Board,
        depth: int,
        current_evaluation: int,
        alpha: int = -Exercise08AI.LIMIT,
        beta: int = Exercise08AI.LIMIT,
        real_depth: int = -1,
    ):
        color = "white" if board.turn else "black"
        ws = (real_depth + 1) * "    "
        key = Exercise04AI.get_key(board, depth)
        if key in self.cache:
            print(
                f"real_depth: {real_depth} {ws} mm> {color}; α: {alpha}; β: {beta}; depth: {depth}"
            )  # memoization
        else:
            print(
                f"real_depth: {real_depth} {ws} --> {color}; α: {alpha}; β: {beta}; depth: {depth}"
            )
        evaluation, best_move, eval_depth = minimax(
            self, board, depth, current_evaluation, alpha, beta, real_depth
        )
        print(
            f"real_depth: {real_depth} {ws} <-- {color}; ev: {evaluation}; ev_depth: {eval_depth}; {best_move}"
        )
        return evaluation, best_move, eval_depth

    return minimax_debug

In [ ]:
class Exercise08AI(Exercise08AI):  # type: ignore
    def push_and_diff(self, board: chess.Board, move: chess.Move) -> int:
        """Pushes the given move on the board and calculates the depth change."""
        if move.promotion:
            board.push(move)
            return 0
        if board.piece_type_at(move.to_square):
            board.push(move)
            return 0
        board.push(move)
        if board.is_check():
            return 0
        return 1

In [ ]:
from typing import Any
import heapq


class Exercise08AI(Exercise08AI):  # type: ignore
    #@debug_minimax
    @Exercise08AI.memoize_minimax
    def minimax(
        self,
        board: chess.Board,
        depth: int,
        current_evaluation: int,
        alpha: int = -Exercise07AI.LIMIT,
        beta: int = Exercise07AI.LIMIT,
        real_depth: int = -1,
    ) -> tuple[int, chess.Move | None, int]:
        """Searches the best value with given depth using minimax algorithm"""
        real_depth += 1
        early_abort_evaluation = self.minimax_early_abort(
            board, depth, current_evaluation
        )
        if early_abort_evaluation is not None:
            self.stats[-1]["leaf_ctr"] += 1
            self.stats[-1]["total_depth"] += real_depth
            return early_abort_evaluation, board.peek(), real_depth

        best_move = None

        # White to play (positive numbers are good)
        moves: list[tuple[int, int, chess.Move]] = []
        if board.turn:
            for i, move in enumerate(board.legal_moves):
                key = Exercise04AI.get_key(board, depth - 2)
                old_eval = self.cache.get(key, (None, i))[1]
                heapq.heappush(moves, (-old_eval, i, move))
            maxEvaluation = -self.LIMIT
            while moves:
                move = heapq.heappop(moves)[2]
                diff = self.push_and_diff(board, move)
                if diff == 0 and real_depth >= self.DEPTH:
                    new_depth = 0
                else:
                    new_depth = depth - diff
                evaluation, _, eval_depth = self.minimax(
                    board,
                    new_depth,
                    self.evaluate(board, current_evaluation),
                    alpha,
                    beta,
                    real_depth,
                )
                board.pop()
                if evaluation >= beta:
                    return evaluation, move, eval_depth
                if evaluation > maxEvaluation or (
                    evaluation == maxEvaluation and best_move is None
                ):
                    best_move = move
                    maxEvaluation = evaluation
                alpha = max(alpha, evaluation)
            return maxEvaluation, best_move, eval_depth

        # Black to play (negative numbers are good)
        else:
            for i, move in enumerate(board.legal_moves):
                key = Exercise04AI.get_key(board, depth - 2)
                old_eval = self.cache.get(key, (None, i))[1]
                heapq.heappush(moves, (old_eval, i, move))
            minEvaluation = self.LIMIT
            while moves:
                move = heapq.heappop(moves)[2]
                diff = self.push_and_diff(board, move)
                if diff == 0 and real_depth >= self.DEPTH:
                    new_depth = 0
                else:
                    new_depth = depth - diff
                evaluation, _, eval_depth = self.minimax(
                    board,
                    new_depth,
                    self.evaluate(board, current_evaluation),
                    alpha,
                    beta,
                    real_depth,
                )
                board.pop()
                if evaluation <= alpha:
                    return evaluation, move, eval_depth
                if evaluation < minEvaluation or (
                    evaluation == minEvaluation and best_move is None
                ):
                    best_move = move
                    minEvaluation = evaluation
                beta = min(beta, evaluation)
            return minEvaluation, best_move, eval_depth

In [ ]:
import sys
import gc

class Exercise08AI(Exercise08AI):  # type: ignore
    def get_next_middle_game_move(self, board: chess.Board) -> chess.Move:
        """Gets the best next move"""
        self.last_evaluation: int | None  # type annotation for mypy
        self.stats[-1]["cache_hits"] = 0
        self.stats[-1]["cache_tries"] = 0
        self.stats[-1]["leaf_ctr"] = 0
        self.stats[-1]["max_depth"] = 0
        self.stats[-1]["total_depth"] = 0

        if self.is_king_endgame != self.check_king_endgame(board):
            self.last_evaluation += self.get_endgame_evaluation_change(board)
        # Calculate current evaluation
        if self.last_evaluation is None:  # type: ignore
            current_evaluation = self.full_evaluate(board)
        else:
            # Get current evaluation (after opponent move)
            current_evaluation = self.evaluate(board, self.last_evaluation)

        for depth in range(1, self.DEPTH + 1):
            # Call minimax and get best move
            future_evaluation, best_move, eval_depth = self.minimax(
                board, depth, current_evaluation
            )

        # Debugging fail save
        assert best_move, f"""
        Best move is None with fen '{board.fen()}' at player {type(self).__name__}! 
        depth: {self.DEPTH}, last_eval: {self.last_evaluation}, current_evaluation: {current_evaluation},
        is_king_engame: {getattr(self, 'is_king_endgame', "N/A")}, move_stack: {board.move_stack}
        """
        # Update last evaluation (after player move)
        self.last_evaluation = current_evaluation + self.incremental_evaluate(
            board, best_move
        )
        # Update stats
        self.stats[-1]["minimax_eval"] = future_evaluation
        self.stats[-1]["board_eval_before_move"] = current_evaluation
        self.stats[-1]["board_eval_after_move"] = self.last_evaluation
        self.stats[-1]["eval_depth"] = eval_depth
        self.stats[-1]["avg_depth"] = (
            self.stats[-1]["total_depth"] / (self.stats[-1]["leaf_ctr"] or 1)
        )

        if board.is_irreversible(best_move):
            self.cache.clear()
            gc.collect()
            self.stats[-1]["cache_cleared"] = True
        else:
            self.stats[-1]["cache_cleared"] = False
        self.stats[-1]["cache_size_mb"] = round(
            sys.getsizeof(self.cache) / (1024 * 1024), 2
        )
        return best_move

## Debugging Bereich

Die folgenden Zellen enthalten Unit-Tests der oben implementierten Funktionen.

In [ ]:
import Exercise03AI as Exercise03AITests
import Exercise04AI as Exercise04AITests

In [ ]:
# Create player and board
unit_test_player = Exercise08AI(player_name="Ex08AI", search_depth=2)
board = chess.Board("5rk1/1b3p2/8/3p4/3p2P1/2Q4B/5P1K/R3R3 b - - 0 36")
board

In [ ]:
# Test minimax
def test_minimax(unit_test_player: ChessAI, board: chess.Board):
    unit_test_player.cache = {}  # Clear cache
    unit_test_player.stats[-1]["cache_tries"] = 0
    unit_test_player.stats[-1]["cache_hits"] = 0
    mm_evaluation, mm_move, mm_eval_depth = unit_test_player.minimax(
        board, depth=2, current_evaluation=1240
    )
    print(f"Minimax Evaluation: {mm_evaluation}")
    print(f"Minimax Move: {mm_move}")
    print(f"Minimax Evaluation Depth: {mm_eval_depth}")
    assert mm_evaluation == 325, "Minimax evaluation does not match expected value!"
    assert mm_move.uci() == "d4c3", "Minimax move does not match expected value!"
    assert unit_test_player.cache != {}, "Cache is empty!"
    print(f"Elements in cache: {len(unit_test_player.cache)}")

In [ ]:
test_minimax(unit_test_player, board)

In [ ]:
# Test next move function (with memoized minimax result)
Exercise03AITests.test_next_move(unit_test_player, board)

## Temporärer Bereich

Der folgende Bereich dient zum temporären Debuggen und kann nicht-funktionierenden Code enthalten. Dieser Bereich wird vor der Abgabe entfernt.